In [15]:
%cd src
!conda run -n simple_env python3 -m numpy.f2py -c -m polyomino_simulation polyomino_simulation.f90 --opt='-O3'
%cd ..

/Users/daviderevignas/scientific_projects/scientific_code/polyomini/src
running build
running config_cc
INFO: unifing config_cc, config, build_clib, build_ext, build commands --compiler options
running config_fc
INFO: unifing config_fc, config, build_clib, build_ext, build commands --fcompiler options
running build_src
INFO: build_src
INFO: building extension "polyomino_simulation" sources
INFO: f2py options: []
INFO: f2py:> /var/folders/n5/f496lkj93z1d5p8bbkfqkhtr0000gn/T/tmps65nmxnp/src.macosx-11.0-arm64-3.9/polyomino_simulationmodule.c
creating /var/folders/n5/f496lkj93z1d5p8bbkfqkhtr0000gn/T/tmps65nmxnp/src.macosx-11.0-arm64-3.9
Reading fortran codes...
	Reading file 'polyomino_simulation.f90' (format:free)
Post-processing...
	Block: polyomino_simulation
			Block: polyomino_simulation
Applying post-processing hooks...
  character_backward_compatibility_hook
Post-processing (stage 2)...
Building modules...
    Building module "polyomino_simulation"...
    Generating possibly empty w

In [16]:
import numpy as np
import matplotlib.pyplot as plt
import os
from numba import njit
import sys 
module_path = "src"
sys.path.append(module_path)
import polyomino_simulation






In [17]:

ominos_sizes= np.array([4,4,4,4,4],dtype=np.int32)
max_omino_size=np.max(ominos_sizes)

omino_1 = np.array([[0,0],[0,1],[0,2],[0,3]],dtype=np.int32)
omino_2 = np.array([[0,0],[0,1],[1,0],[1,1]],dtype=np.int32)
omino_3 = np.array([[0,0],[0,1],[0,2],[1,2]],dtype=np.int32)
omino_4 = np.array([[0,0],[0,1],[0,2],[1,1]],dtype=np.int32)
omino_5 = np.array([[0,0],[0,1],[1,1],[1,2]],dtype=np.int32)

omino_1 = omino_1[..., np.newaxis]  # (4,2,1)
omino_2 = omino_2[..., np.newaxis] 
omino_3 = omino_3[..., np.newaxis]  
omino_4 = omino_4[..., np.newaxis]  
omino_5 = omino_5[..., np.newaxis]  

omino_shapes = np.concatenate([omino_1, omino_2, omino_3, omino_4,omino_5], axis=2)

n_omino_types = omino_shapes.shape[-1]

n_ominos_per_shape=np.array([1,1,1,1,1],dtype=np.int32)
box_size = 25

n_ominos = np.sum(n_ominos_per_shape)


In [18]:
n_lattice_sites = box_size**2
initial_positions = np.array([[1,1],[3,1],[6,6],[9,9],[12,12]],dtype=np.int32)
initial_orientations = np.array([1,1,1,1,1],dtype=np.int32)


In [19]:
n_mc_steps=800 
n_savings=100
prng_seed=np.random.randint(1,100000)

system_history=polyomino_simulation.polyomino_simulation(
    n_mc_steps=n_mc_steps,
    n_savings=n_savings,
    n_omino_types=n_omino_types,
    ominos_sizes=ominos_sizes,
    max_omino_size=max_omino_size,
    omino_shapes=omino_shapes,
    n_ominos_per_shape=n_ominos_per_shape,
    n_ominos=n_ominos,
    initial_positions=initial_positions,
    initial_orientations=initial_orientations,
    box_size=box_size,
    prng_seed=prng_seed,
)

           1           2           3           4           5


In [20]:
system_history.sum()

2000

In [21]:
system_history.shape

(25, 25, 100)

In [22]:
# fig,ax=plt.subplots()
# ax.set_aspect('equal')
# ax.set_axisbelow(True)

# i_snapshot=0

# ax.set_xlim([0,box_size+1])
# ax.set_ylim([0,box_size+1])

# for i in range(box_size + 1):
#     ax.plot([i+0.5, i+0.5],[0.5, box_size+0.5], color='black')
#     ax.plot([0.5, box_size+0.5],[i+0.5, i+0.5], color='black')


# block_size_plot=17
# for ix in range(box_size):
#     for iy in range(box_size):
#         if system_history[ix,iy,i_snapshot]>0:
#             ax.plot(ix+1,iy+1,'sk',markersize=block_size_plot,markeredgewidth=0.0)


In [23]:
import matplotlib.pyplot as plt
import matplotlib.animation as animation

# --- Setup ---
fig, ax = plt.subplots()
ax.set_aspect('equal')
ax.set_axisbelow(True)

ax.set_xlim([0, box_size+1])
ax.set_ylim([0, box_size+1])

# Draw grid once (static)
for i in range(box_size + 1):
    ax.plot([i+0.5, i+0.5], [0.5, box_size+0.5], color='black')
    ax.plot([0.5, box_size+0.5], [i+0.5, i+0.5], color='black')

block_size_plot = 14
scatters = []  # keep references for clearing


# --- Update function for animation ---
def update(i_snapshot):
    global scatters
    # remove old markers
    for sc in scatters:
        sc.remove()
    scatters = []

    for ix in range(box_size):
        for iy in range(box_size):
            if system_history[ix, iy, i_snapshot] > 0:
                sc = ax.plot(ix+1, iy+1, 'sk',
                             markersize=block_size_plot,
                             markeredgewidth=0.0)
                scatters.extend(sc)  # plot returns a list

    return scatters


# --- Create animation ---
n_snapshots = system_history.shape[2]  # number of frames
ani = animation.FuncAnimation(fig, update, frames=n_snapshots, blit=False, repeat=True)

# --- Save as GIF ---
ani.save("animation.gif", writer="pillow", fps=5)
plt.close()
